In [2]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import ipywidgets as widgets
import pandas as pd
import pickle
import numpy as np
import os

# Batch run data

In [17]:
# batch run values

n_vals = 2

N = 700
num_ideas = 700
n_ticks = 120# number of half years the model should run




variable_params = {'N' : N, 
                   'num_ideas' : num_ideas ,
                   'mean_ideology' : [0.05,0.5] ,
                   'error' : [0.1,0.3] ,
                   'p_false_positive' : np.linspace(0.05, 0.8, n_vals) ,
                   'p_false_negative' : np.linspace(0.05, 0.8, n_vals) ,
                   'p_true_start' : 0.9, 
                   'p_new_idea' : [0.3,0.7],
                   'p_new_idea_true' : [0.3,0.7],
                   'p_ideology_idea' : np.linspace(0.1, 0.8, n_vals),
                   'sd_ideology': [0.2,1],
                   'promotion_weight_ideology' : [0.3,0.7],
                   'p_ideology_reviewer' : np.linspace(0.1,0.8, n_vals),
                  'censure': [True,False],
                  'sd_ideas' : [0.2,1] 
}

In [10]:
# batch run data
# have to recombine the 25mb chunks into a single file due to github size limits



def recombine_parts(parts, output_file_path):
    """Recombine parts back into a single file, checking the integrity of each part."""
    with open(output_file_path, 'wb') as output_file:
        for part_file_path in parts:
            if not os.path.exists(part_file_path):
                print(f"Missing part: {part_file_path}")
                continue
            with open(part_file_path, 'rb') as part_file:
                part_data = part_file.read()
                if not part_data:
                    print(f"No data could be read from {part_file_path}")
                output_file.write(part_data)
                print(f'Part {part_file_path} added, size {len(part_data)} bytes.')
    print('All parts have been recombined.')
    
parts = ['part_0.pkl',
 'part_1.pkl',
 'part_2.pkl',
 'part_3.pkl',
 'part_4.pkl',
 'part_5.pkl',
 'part_6.pkl']

# # Example usage to recombine and then load the data
recombine_parts(parts, output_file_path='recombined.pkl')  # set num_parts correctly based on how many parts you have

# Load the original data object
with open('recombined.pkl', 'rb') as file:
    batch_run = pickle.load(file)

df = pd.DataFrame(batch_run)

Part part_0.pkl added, size 26214400 bytes.
Part part_1.pkl added, size 26214400 bytes.
Part part_2.pkl added, size 26214400 bytes.
Part part_3.pkl added, size 26214400 bytes.
Part part_4.pkl added, size 26214400 bytes.
Part part_5.pkl added, size 26214400 bytes.
Part part_6.pkl added, size 12402944 bytes.
All parts have been recombined.


# Plotting code

In [15]:
# Plotting functions

def preprocess_data(filtered_df):
    # Make a copy of the DataFrame to avoid SettingWithCopyWarning
    df = filtered_df.copy()

    df['true_ideas'] = df[['True Ideas < -1', 'True Ideas -1 to 0', 'True Ideas 0 to 1', 'True Ideas > 1']].sum(axis=1)
    df['false_ideas'] = df[['False Ideas < -1', 'False Ideas -1 to 0', 'False Ideas 0 to 1', 'False Ideas > 1']].sum(axis=1)
    df['con'] = df['True Ideas -1 to 0'] + df['False Ideas -1 to 0']
    df['e_con'] = df['True Ideas < -1'] + df['False Ideas < -1']
    df['lib'] = df['True Ideas 0 to 1'] + df['False Ideas 0 to 1']
    df['e_lib'] = df['True Ideas > 1'] + df['False Ideas > 1']
    df['total_researchers'] = df['Mean Orientation Tenured'] + df['Mean Orientation Candidate']

    return df


def update_plots(p_false_positive, p_false_negative, p_ideology_idea, promotion_weight_ideology, p_ideology_reviewer,
                 censure, mean_ideology,error,p_new_idea,p_new_idea_true, sd_ideology,sd_ideas): #
    filtered_df = df[
        (df['p_false_positive'] == p_false_positive) &
        (df['p_false_negative'] == p_false_negative) &
        (df['p_ideology_idea'] == p_ideology_idea) &
        (df['promotion_weight_ideology'] == promotion_weight_ideology) &
        (df['p_ideology_reviewer'] == p_ideology_reviewer)  &
        (df['censure'] == censure) &
        (df['mean_ideology'] == mean_ideology) &
        (df['error'] == error) &
        (df['p_new_idea'] == p_new_idea) &
        (df['p_new_idea_true'] == p_new_idea_true) &
        (df['sd_ideology'] == sd_ideology) &
        (df['sd_ideas'] == sd_ideas)
        
    ] #

    if filtered_df.empty:
        print("No data available for this combination of parameters.")
        return

    filtered_df = preprocess_data(filtered_df)
    n_ticks = len(filtered_df)  # Assuming the number of ticks is the length of the dataframe

    # Plotting as per your code with slight modifications for clarity and correctness
    fig, axs = plt.subplots(2, 3, figsize=(18, 12))
    axs = axs.flatten()

    
    axs[0].plot(filtered_df['Step'],filtered_df['Researchers < -1'], color = 'darkblue')
    axs[0].plot(filtered_df['Step'],filtered_df['Researchers -1 to 0'], color = 'lightblue')
    axs[0].plot(filtered_df['Step'],filtered_df['Researchers 0 to 1'], color = 'lightgreen')
    axs[0].plot(filtered_df['Step'],filtered_df['Researchers > 1'], color = 'darkgreen')
    axs[0].set_title('Number of researchers')
    

    
    # Number of ideas over time
    axs[1].plot(filtered_df['Step'], filtered_df['e_con'], label='Extreme conservative', color='darkblue')
    axs[1].plot(filtered_df['Step'], filtered_df['con'], label='Conservative', color='lightblue')
    axs[1].plot(filtered_df['Step'], filtered_df['lib'], label='Liberal', color='lightgreen')
    axs[1].plot(filtered_df['Step'], filtered_df['e_lib'], label='Extreme liberal', color='darkgreen')
    axs[1].plot(filtered_df['Step'], filtered_df['total neutral ideas'], label='Neutral', color='grey')
    axs[1].set_title('Number of Ideas')
    axs[1].legend()

    # Ideological orientation over time
    axs[2].plot(filtered_df['Step'], filtered_df['Mean Orientation Tenured'], label='Tenured Researchers', color='red')
    axs[2].plot(filtered_df['Step'], filtered_df['Mean Orientation Accepted Ideas'], label='Accepted Ideas', color='grey')
    axs[2].set_title('Mean Ideological Orientation')
    axs[2].legend()

    # Truth value of ideas
    axs[3].plot(filtered_df['Step'], filtered_df['False Ideas'], label='False Ideas', color='red')
    axs[3].plot(filtered_df['Step'], filtered_df['True Ideas'], label='True Ideas', color='green')
    axs[3].set_title('Truth Value')
    axs[3].legend()

    # True and False Ideas by Ideology
    axs[4].plot(filtered_df['Step'], filtered_df['True Ideas > 1'] + filtered_df['True Ideas 0 to 1'], label='True Liberal', color='green')
    axs[4].plot(filtered_df['Step'], filtered_df['False Ideas > 1'] + filtered_df['False Ideas 0 to 1'], label='False Liberal', color='green', linestyle=':')
    axs[4].plot(filtered_df['Step'], filtered_df['True Ideas < -1'] + filtered_df['True Ideas -1 to 0'], label='True Conservative', color='blue')
    axs[4].plot(filtered_df['Step'], filtered_df['False Ideas < -1'] + filtered_df['False Ideas -1 to 0'], label='False Conservative', color='blue', linestyle=':')
    axs[4].plot(filtered_df['Step'],filtered_df['total neutral ideas'] - filtered_df['total true neutral ideas'], label = 'False neutral ideas', color = 'grey', linestyle = ':')
    axs[4].plot(filtered_df['Step'],filtered_df['total true neutral ideas'], label = 'True neutral ideas', color = 'grey')
    axs[4].set_title('True and False Ideas by Ideology')
    axs[4].legend()
    



    # Proportion of Accepted Ideas
    axs[5].plot(filtered_df['Step'], filtered_df['Proportion Accepted'], label='Proportion of ideas', color='purple')
    axs[5].set_title('Proportion of Falsely Accepted and Rejected Ideas Over Time')
    axs[5].legend()

    plt.tight_layout()
    plt.show()




In [18]:
# plottig code
style = {'description_width': 'initial'}
slider_layout = widgets.Layout(width='350px')  # Adjust width as needed    
p_false_positive_slider = widgets.SelectionSlider(options=[(f"{x:.2f}", x) for x in np.linspace(0.05, 0.8, n_vals)], value=0.05, description='p_false_positive',style=style, layout = slider_layout)
p_false_negative_slider = widgets.SelectionSlider(options=[(f"{x:.2f}", x) for x in np.linspace(0.05, 0.8, n_vals)], value=0.05, description='p_false_negative',style=style, layout = slider_layout)
p_ideology_idea_slider = widgets.SelectionSlider(options=[(f"{x:.2f}", x) for x in np.linspace(0.1, 0.8, n_vals)], value=0.1, description='p_ideology_idea',style=style, layout = slider_layout)
promotion_weight_ideology_slider = widgets.SelectionSlider(options=[(f"{x:.1f}", x) for x in [0.3, 0.7]], value=0.3, description='Promotion Weight Ideology',style=style, layout = slider_layout)
p_ideology_reviewer_slider = widgets.SelectionSlider(options=[(f"{x:.2f}", x) for x in np.linspace(0.1, 0.8, n_vals)], value=0.1, description='p_ideology_reviewer',style=style, layout = slider_layout)
censure_toggle = widgets.ToggleButtons(options=[(str(x), x) for x in [True, False]], value=True, description='Censure')

mean_ideology_slider = widgets.SelectionSlider(options= [0.05,0.5] , value=0.05, description='mean_ideology',style=style, layout = slider_layout)
error_slider = widgets.SelectionSlider(options=[0.1,0.3] , value=0.1, description='error',style=style, layout = slider_layout)
p_new_idea_slider = widgets.SelectionSlider(options=[0.3,0.7], value=0.3, description='p_new_idea',style=style, layout = slider_layout)
p_new_idea_true_slider = widgets.SelectionSlider(options=[0.3,0.7], value=0.3, description='p_new_idea_true',style=style, layout = slider_layout)
sd_ideology_slider = widgets.SelectionSlider(options=[0.2,1], value=0.2, description='sd_ideology',style=style, layout = slider_layout)
sd_ideas_slider = widgets.SelectionSlider(options=[0.2,1], value=0.2, description='sd_ideas',style=style, layout = slider_layout)




ui = widgets.VBox([p_false_positive_slider, p_false_negative_slider, p_ideology_idea_slider, promotion_weight_ideology_slider, p_ideology_reviewer_slider,
                  mean_ideology_slider, error_slider, p_new_idea_slider, p_new_idea_true_slider, sd_ideology_slider ,sd_ideas_slider, censure_toggle]) #
out = widgets.interactive_output(update_plots, {'p_false_positive': p_false_positive_slider, 'p_false_negative': p_false_negative_slider, 'p_ideology_idea': p_ideology_idea_slider, 
                                                'promotion_weight_ideology': promotion_weight_ideology_slider,'p_ideology_reviewer': p_ideology_reviewer_slider, 
                                                 'mean_ideology':mean_ideology_slider, 'error':error_slider,
                                               'p_new_idea':p_new_idea_slider, 'p_new_idea_true':p_new_idea_true_slider, 'sd_ideology':sd_ideology_slider,
                                               'sd_ideas':sd_ideas_slider,'censure': censure_toggle}) #



# Display

In [19]:
# display

display(ui, out)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 1800x1200 with 6 Axes>', …